In [126]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import  r2_score

# Veri yükleme
df = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')

# Kullanılmayan sütunları belirleme 
unused_columns = ['Id']  
df = df.drop(columns=unused_columns)

# Kategorik ve sayısal sütunları ayırma
numeric_features = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd']
categorical_features = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle']

# Ön işleme adımı
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Eksik sayısal verileri ortalama ile doldurma
            ('scaler', StandardScaler())  # Sayısal verileri standardize etme
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Eksik kategorik verileri mod ile doldurma
            ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Kategorik verileri encode etme
        ]), categorical_features)
    ])

# Modeli pipeline içine yerleştirme
model = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Ön işleme adımını ekleme
    ('regressor', GradientBoostingRegressor(n_estimators=1000, random_state=42))  # Modeli ekleme
])

# Bağımsız (X) ve bağımlı (y) değişkenleri ayırma
X = df.drop('SalePrice', axis=1)  # 'SalePrice' haricindeki tüm sütunlar X
y = df['SalePrice']  # 'SalePrice', y

# Eğitim ve test setlerini ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model eğitimi
model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
predictions = model.predict(X_test)

# Modelin başarımı
r2 = r2_score(y_test, predictions)


print(f'R^2 Score: {r2}')



R^2 Score: 0.8420727672881064


In [122]:
new_data = {
    'LotFrontage': [80],  # Örnek veriler
    'LotArea': [10000],
    'OverallQual': [7],
    'OverallCond': [5],
    'YearBuilt': [1995],
    'YearRemodAdd': [2015],
    'MSZoning': ['RL'],
    'Street': ['Pave'],
    'LotShape': ['Reg'],
    'LandContour': ['Lvl'],
    'Utilities': ['AllPub'],
    'LotConfig': ['Inside'],
    'LandSlope': ['Gtl'],
    'Neighborhood': ['CollgCr'],
    'Condition1': ['Norm'],
    'Condition2': ['Norm'],
    'BldgType': ['1Fam'],
    'HouseStyle': ['2Story']
}

# Yeni veriyi DataFrame olarak oluşturma
X_new = pd.DataFrame(new_data)

# Yeni verilerle tahmin yapma
new_predictions = model.predict(X_new)

# Tahminleri tam sayıya yuvarlama
new_predictions_rounded = new_predictions.round()

print("Yeni Veriler İçin Tahminler:", new_predictions_rounded)


Yeni Veriler İçin Tahminler: [246728.]


In [118]:
import pickle

# Modeli kaydetme
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
